In [1]:
# Trying to create the Mahalonobis Distance and the other measures from the Bloom Paper 
import pandas as pd 
import numpy as np
import os

In [3]:
patents = pd.read_stata('H:/InnovationProject/data/temp/new_dataset3_techspill.dta')
patents.head()

,patnum,czone,ipc_group,assignee_id
0,3930273,24701,87,126430.0
1,3930294,11600,156,60899.0
2,3930298,16000,153,98565.0
3,3930298,16000,155,98565.0
4,3930357,20401,331,132586.0


In [2]:
patents = pd.read_stata('H:/InnovationProject/data/temp/new_dataset3_techspill.dta')
patents = patents.groupby(['assignee_id', 'ipc_group', 'czone'])['patnum'].nunique().reset_index()
patents.columns = ['assignee_id', 'ipc4', 'czone', 'ipc_patents']
patents['total_patents'] = patents.groupby(['assignee_id', 'czone'])['ipc_patents'].transform('sum')
patents['share'] = patents['ipc_patents']/patents['total_patents']

In [12]:
data_directory = 'H:/InnovationProject/data/temp/czone'
unique_cz_values = patents['czone'].unique()
# Removing Silicon Valley because this is just too big 
new_cz_values = [value for value in unique_cz_values if value not in [19600, 33100, 37800, 20500, 38300]]
new_cz_values
len(new_cz_values)

715

In [11]:
for cz in new_cz_values[24:]:
    
    print(cz)
    cz_rows = patents[patents['czone'] == cz]
    temp = cz_rows.copy()
    T = temp.pivot(index = 'ipc4', columns = 'assignee_id', values = 'share')
    T = T.fillna(0)

    X_t = temp.pivot(index = 'assignee_id', columns ='ipc4', values = 'share')
    X_t = X_t.fillna(0)
    T_t = T.copy()
    
    for column in list(T_t): 
        T_t[column] = T_t[column] /(np.sqrt(np.dot(T_t[[column]].values.T, T_t[[column]].values)[0][0]))
        
    om_f =X_t.T.dot(X_t)
    om_s =X_t.T.dot(X_t)
    for sic1 in list(om_s): 
        for sic2 in list(om_s): 
            om_s.loc[sic1][sic2] = om_s.loc[sic1][sic2]/(np.sqrt(om_f[sic1][sic1]) * np.sqrt(om_f[sic2][sic2]))
    # Now creating similar values as in the Stata code by Bloom 
    maltech = T_t.T.dot(om_s).dot(T_t)
    malcovtech = X_t.dot(om_s).dot(X_t.T)
    tech = T_t.T.dot(T_t)
    covtech = X_t.dot(X_t.T)
        
    #Mahalanobis Closeness in Technology Space (TECH)
    maltech = maltech.unstack().reset_index(level=1)
    maltech.columns = ['assignee_id_pat', 'maltec']
    maltech = maltech.reset_index()
    maltech = maltech[(maltech['assignee_id'] != maltech['assignee_id_pat'])].sort_values(['assignee_id', 'assignee_id_pat']).reset_index(drop = True)
        
    #Mahalanobis Covariance Closeness in Technology Space (TECH)
    malcovtech = malcovtech.unstack().reset_index(level=1)
    malcovtech.columns = ['assignee_id_pat', 'malcovtec']
    malcovtech = malcovtech.reset_index()
    malcovtech = malcovtech[(malcovtech['assignee_id'] != malcovtech['assignee_id_pat'])].sort_values(['assignee_id', 'assignee_id_pat']).reset_index(drop = True)
        
    #Covariance Closeness in Technology Space 
    covtech = covtech.unstack().reset_index(level=1)
    covtech.columns = ['assignee_id_pat', 'covtec']
    covtech = covtech.reset_index()
    covtech = covtech[(covtech['assignee_id'] != covtech['assignee_id_pat'])].sort_values(['assignee_id', 'assignee_id_pat']).reset_index(drop = True)
        
    # Jaffe Index 
    tech = tech.unstack().reset_index(level=1)
    tech.columns = ['assignee_id_pat', 'tec']
    tech = tech.reset_index()
    tech = tech[(tech['assignee_id'] != tech['assignee_id_pat'])].sort_values(['assignee_id', 'assignee_id_pat']).reset_index(drop = True)
    tech.head()
        
    merged_df1 = pd.merge(maltech, malcovtech, on =['assignee_id', 'assignee_id_pat'])
    merged_df2 = pd.merge(tech, covtech, on =['assignee_id', 'assignee_id_pat'])
    merged_df3 = pd.merge(merged_df1, merged_df2, on =['assignee_id', 'assignee_id_pat'])
        
    finalfilename = os.path.join(data_directory,f'technology{cz}.dta')
    print(finalfilename)
    merged_df3.to_stata(finalfilename)

24100
H:/InnovationProject/data/temp/czone\technology24100.dta
38000
H:/InnovationProject/data/temp/czone\technology38000.dta
6900
H:/InnovationProject/data/temp/czone\technology6900.dta
7000
H:/InnovationProject/data/temp/czone\technology7000.dta
15800
H:/InnovationProject/data/temp/czone\technology15800.dta
18900
H:/InnovationProject/data/temp/czone\technology18900.dta
34901
H:/InnovationProject/data/temp/czone\technology34901.dta
15900
H:/InnovationProject/data/temp/czone\technology15900.dta
20600
H:/InnovationProject/data/temp/czone\technology20600.dta
39400
H:/InnovationProject/data/temp/czone\technology39400.dta
11304
H:/InnovationProject/data/temp/czone\technology11304.dta
19700
H:/InnovationProject/data/temp/czone\technology19700.dta
36100
H:/InnovationProject/data/temp/czone\technology36100.dta
36000
H:/InnovationProject/data/temp/czone\technology36000.dta
9100
H:/InnovationProject/data/temp/czone\technology9100.dta
10700
H:/InnovationProject/data/temp/czone\technology10700.dt

H:/InnovationProject/data/temp/czone\technology15300.dta
4103
H:/InnovationProject/data/temp/czone\technology4103.dta
10900
H:/InnovationProject/data/temp/czone\technology10900.dta
28800
H:/InnovationProject/data/temp/czone\technology28800.dta
23801
H:/InnovationProject/data/temp/czone\technology23801.dta
29700
H:/InnovationProject/data/temp/czone\technology29700.dta
29601
H:/InnovationProject/data/temp/czone\technology29601.dta
10502
H:/InnovationProject/data/temp/czone\technology10502.dta
26412
H:/InnovationProject/data/temp/czone\technology26412.dta
37604
H:/InnovationProject/data/temp/czone\technology37604.dta
9400
H:/InnovationProject/data/temp/czone\technology9400.dta
17300
H:/InnovationProject/data/temp/czone\technology17300.dta
38601
H:/InnovationProject/data/temp/czone\technology38601.dta
33803
H:/InnovationProject/data/temp/czone\technology33803.dta
31800
H:/InnovationProject/data/temp/czone\technology31800.dta
7900
H:/InnovationProject/data/temp/czone\technology7900.dta
1880

18201
H:/InnovationProject/data/temp/czone\technology18201.dta
39302
H:/InnovationProject/data/temp/czone\technology39302.dta
26803
H:/InnovationProject/data/temp/czone\technology26803.dta
12402
H:/InnovationProject/data/temp/czone\technology12402.dta
8201
H:/InnovationProject/data/temp/czone\technology8201.dta
14700
H:/InnovationProject/data/temp/czone\technology14700.dta
8100
H:/InnovationProject/data/temp/czone\technology8100.dta
5401
H:/InnovationProject/data/temp/czone\technology5401.dta
8000
H:/InnovationProject/data/temp/czone\technology8000.dta
25601
H:/InnovationProject/data/temp/czone\technology25601.dta
16701
H:/InnovationProject/data/temp/czone\technology16701.dta
35701
H:/InnovationProject/data/temp/czone\technology35701.dta
36404
H:/InnovationProject/data/temp/czone\technology36404.dta
29403
H:/InnovationProject/data/temp/czone\technology29403.dta
19902
H:/InnovationProject/data/temp/czone\technology19902.dta
31102
H:/InnovationProject/data/temp/czone\technology31102.dta


H:/InnovationProject/data/temp/czone\technology6402.dta
8900
H:/InnovationProject/data/temp/czone\technology8900.dta
9001
H:/InnovationProject/data/temp/czone\technology9001.dta
9003
H:/InnovationProject/data/temp/czone\technology9003.dta
5700
H:/InnovationProject/data/temp/czone\technology5700.dta
35904
H:/InnovationProject/data/temp/czone\technology35904.dta
34301
H:/InnovationProject/data/temp/czone\technology34301.dta
18700
H:/InnovationProject/data/temp/czone\technology18700.dta
14802
H:/InnovationProject/data/temp/czone\technology14802.dta
12401
H:/InnovationProject/data/temp/czone\technology12401.dta
35402
H:/InnovationProject/data/temp/czone\technology35402.dta
35902
H:/InnovationProject/data/temp/czone\technology35902.dta
2900
H:/InnovationProject/data/temp/czone\technology2900.dta
25103
H:/InnovationProject/data/temp/czone\technology25103.dta
26901
H:/InnovationProject/data/temp/czone\technology26901.dta
23301
H:/InnovationProject/data/temp/czone\technology23301.dta
8702
H:/I

H:/InnovationProject/data/temp/czone\technology20403.dta
29506
H:/InnovationProject/data/temp/czone\technology29506.dta
20003
H:/InnovationProject/data/temp/czone\technology20003.dta
27101
H:/InnovationProject/data/temp/czone\technology27101.dta
4004
H:/InnovationProject/data/temp/czone\technology4004.dta
28201
H:/InnovationProject/data/temp/czone\technology28201.dta
25300
H:/InnovationProject/data/temp/czone\technology25300.dta
34304
H:/InnovationProject/data/temp/czone\technology34304.dta
31001
H:/InnovationProject/data/temp/czone\technology31001.dta
29003
H:/InnovationProject/data/temp/czone\technology29003.dta
28601
H:/InnovationProject/data/temp/czone\technology28601.dta
30501
H:/InnovationProject/data/temp/czone\technology30501.dta
33601
H:/InnovationProject/data/temp/czone\technology33601.dta
26001
H:/InnovationProject/data/temp/czone\technology26001.dta
21801
H:/InnovationProject/data/temp/czone\technology21801.dta
27401
H:/InnovationProject/data/temp/czone\technology27401.dta


31004
H:/InnovationProject/data/temp/czone\technology31004.dta
34307
H:/InnovationProject/data/temp/czone\technology34307.dta
34309
H:/InnovationProject/data/temp/czone\technology34309.dta
26103
H:/InnovationProject/data/temp/czone\technology26103.dta
28606
H:/InnovationProject/data/temp/czone\technology28606.dta
37902
H:/InnovationProject/data/temp/czone\technology37902.dta
34103
H:/InnovationProject/data/temp/czone\technology34103.dta
30902
H:/InnovationProject/data/temp/czone\technology30902.dta
3101
H:/InnovationProject/data/temp/czone\technology3101.dta
39204
H:/InnovationProject/data/temp/czone\technology39204.dta
28002
H:/InnovationProject/data/temp/czone\technology28002.dta
31304
H:/InnovationProject/data/temp/czone\technology31304.dta
32304
H:/InnovationProject/data/temp/czone\technology32304.dta
35905
H:/InnovationProject/data/temp/czone\technology35905.dta
32402
H:/InnovationProject/data/temp/czone\technology32402.dta
29007
H:/InnovationProject/data/temp/czone\technology2900

In [13]:
# Try and create seperate files for the commuting zones for which I had a problem 

large_cz_values = [19600, 33100, 37800, 20500, 38300]

In [14]:
data_directory = 'H:/InnovationProject/data/temp/czone/special_czones'

for cz in large_cz_values:
    
    print(cz)
    cz_rows = patents[patents['czone'] == cz]
    temp = cz_rows.copy()
    T = temp.pivot(index = 'ipc4', columns = 'assignee_id', values = 'share')
    T = T.fillna(0)

    X_t = temp.pivot(index = 'assignee_id', columns ='ipc4', values = 'share')
    X_t = X_t.fillna(0)
    T_t = T.copy()
    
    for column in list(T_t): 
        T_t[column] = T_t[column] /(np.sqrt(np.dot(T_t[[column]].values.T, T_t[[column]].values)[0][0]))
        
    om_f =X_t.T.dot(X_t)
    om_s =X_t.T.dot(X_t)
    for sic1 in list(om_s): 
        for sic2 in list(om_s): 
            om_s.loc[sic1][sic2] = om_s.loc[sic1][sic2]/(np.sqrt(om_f[sic1][sic1]) * np.sqrt(om_f[sic2][sic2]))
            
            
    # Now creating similar values as in the Stata code by Bloom 
    tech = T_t.T.dot(om_s).dot(T_t)
    
    #Mahalanobis Closeness in Technology Space (TECH)
    tech = tech.unstack().reset_index(level=1)
    tech.columns = ['assignee_id_pat', 'maltec']
    tech = tech.reset_index()
    tech = tech[(tech['assignee_id'] != tech['assignee_id_pat'])].sort_values(['assignee_id', 'assignee_id_pat']).reset_index(drop = True)
    
    finalfilename = os.path.join(data_directory,f'maltec{cz}.dta')
    print(finalfilename)
    tech.to_stata(finalfilename)
        
    #Mahalanobis Covariance Closeness in Technology Space (TECH)
    tech = X_t.dot(om_s).dot(X_t.T)
    
    tech = tech.unstack().reset_index(level=1)
    tech.columns = ['assignee_id_pat', 'malcovtec']
    tech = tech.reset_index()
    tech = tech[(tech['assignee_id'] != tech['assignee_id_pat'])].sort_values(['assignee_id', 'assignee_id_pat']).reset_index(drop = True)
    
    finalfilename = os.path.join(data_directory,f'malcovtec{cz}.dta')
    print(finalfilename)
    tech.to_stata(finalfilename)
        
    #Covariance Closeness in Technology Space
    tech = X_t.dot(X_t.T)
 
    tech = tech.unstack().reset_index(level=1)
    tech.columns = ['assignee_id_pat', 'covtec']
    tech = tech.reset_index()
    tech = tech[(tech['assignee_id'] != tech['assignee_id_pat'])].sort_values(['assignee_id', 'assignee_id_pat']).reset_index(drop = True)
 
    finalfilename = os.path.join(data_directory,f'covtec{cz}.dta')
    print(finalfilename)
    tech.to_stata(finalfilename)
    
    # Jaffe Index 
    tech = T_t.T.dot(T_t)
    
    tech = tech.unstack().reset_index(level=1)
    tech.columns = ['assignee_id_pat', 'tec']
    tech = tech.reset_index()
    tech = tech[(tech['assignee_id'] != tech['assignee_id_pat'])].sort_values(['assignee_id', 'assignee_id_pat']).reset_index(drop = True)
    
    finalfilename = os.path.join(data_directory,f'tec{cz}.dta')
    print(finalfilename)
    tech.to_stata(finalfilename)

19600
H:/InnovationProject/data/temp/czone/special_czones\maltec19600.dta
H:/InnovationProject/data/temp/czone/special_czones\malcovtec19600.dta
H:/InnovationProject/data/temp/czone/special_czones\covtec19600.dta
H:/InnovationProject/data/temp/czone/special_czones\tec19600.dta
33100
H:/InnovationProject/data/temp/czone/special_czones\maltec33100.dta
H:/InnovationProject/data/temp/czone/special_czones\malcovtec33100.dta
H:/InnovationProject/data/temp/czone/special_czones\covtec33100.dta
H:/InnovationProject/data/temp/czone/special_czones\tec33100.dta
37800
H:/InnovationProject/data/temp/czone/special_czones\maltec37800.dta
H:/InnovationProject/data/temp/czone/special_czones\malcovtec37800.dta
H:/InnovationProject/data/temp/czone/special_czones\covtec37800.dta
H:/InnovationProject/data/temp/czone/special_czones\tec37800.dta
20500
H:/InnovationProject/data/temp/czone/special_czones\maltec20500.dta
H:/InnovationProject/data/temp/czone/special_czones\malcovtec20500.dta
H:/InnovationProject/